In [5]:
#2017/3/21 程度詞及後續詞的觀察
#2017/3/25 增加程度詞後搜尋
#2017/3/26 建立各種搜尋，包含印出
#資料庫：平衡語料庫
#找尋範圍：程度詞後1~2個
import codecs
import os
from collections import defaultdict
import time
from IPython.display import clear_output

SC_path = "D:\\課業相關\\論文資料\\SCS2\\"
out_path = "C:\\Users\\user\\Desktop\\"

condicate = ['太','最','更','比較','很','稍','略','多','挺','非常','十分','稍微','多麼','極','好','怪','稍稍',\
             '不大','格外','極其','頂','越發']
new = ['甚','甚為','何等','至','至為','些微','益發','稍許','何其','不甚','好生','殊']

In [6]:
#找出程度詞後1~2個接續詞
dfa_dict = defaultdict(int)
dfa_num = defaultdict(int)

for i in condicate: #確保候選都有出現在陣列內
    dfa_num[i] = 0

for index,file in enumerate(os.listdir(SC_path)):
    
    if index % 1000 == 0:
        print (index)
    
    with codecs.open(SC_path+file,'rb','utf8') as f:
        header = f.readline()
        content = f.readline().strip().split()
        
        for i in range(len(content)-2):
            if content[i].split('(')[0] in condicate and 'CATEGORY' not in content[i+1].split('(')[1] \
                and 'CATEGORY' not in content[i+2].split('(')[1]:
                dfa_dict[content[i]+content[i+1]] += 1
                dfa_dict[content[i]+content[i+1]+content[i+2]] += 1
                dfa_num[content[i].split('(')[0]] += 1
        if len(content) >= 2 and content[-2].split('(')[0] in condicate and 'CATEGORY' not in content[-1].split('(')[1]:
            dfa_dict[content[-2]+content[-1]] += 1
            dfa_num[content[-2].split('(')[0]] += 1

clear_output()
            
dfa_ans = sorted(dfa_dict.items(), key=lambda d:d[1], reverse = True)
dfa_nums = sorted(dfa_num.items(), key=lambda d:d[1], reverse = True)

for i in dfa_nums:
    print (i[0],i[1])
    
with codecs.open(out_path+'dfa_sort.csv','wb','utf8') as g:
    for i in dfa_ans:
        g.write(i[0]+','+str(i[1])+'\r\n')
        
print ('END')

至 5521
甚 259
何等 35
些微 34
殊 28
何其 25
益發 19
甚為 17
至為 4
好生 4
稍許 3
不甚 0
END


In [7]:
#建立程度詞搜尋表
input_path = out_path

sort_word_fre = []
sort_pos_fre = defaultdict(int)
pos_word_set = defaultdict(dict) #用詞性找包含該詞性的詞彙頻率
word_word_set = defaultdict(dict) #用詞彙找包含該詞彙的詞彙頻率

def pos_set(pos): #找出詞性所有組合
    temp_count = len(pos.split(')'))-1 #計算為多少個詞性
    temp_set = []
    for i in range(0,temp_count): #調整最少從幾位開始
        temp_set.append(')'.join(pos.split(')')[:i+1])+')')
    return temp_set

with codecs.open(input_path+'dfa_sort.csv','rb','utf8') as f:
    content = f.readlines()
    
    for line in content:
        line = line.strip()
        if line != '':
            line = line.split(',')
            sort_word_fre.append((line[0],int(line[1])))
            
            pos_temp = ''
            for i in line[0].split(')'):
                if '(' in i:
                    pos_temp += '('+i.split('(')[1]+')'
            sort_pos_fre[pos_temp] += int(line[1])
            
            temp_set = pos_set(pos_temp)
            for i in temp_set:
                if line[0] not in pos_word_set[i]:
                    pos_word_set[i][line[0]] = int(line[1])
                else:
                    pos_word_set[i][line[0]] += int(line[1])
                    
            temp_set = pos_set(line[0])
            for i in temp_set:
                if line[0] not in word_word_set[i]:
                    word_word_set[i][line[0]] = int(line[1])
                else:
                    word_word_set[i][line[0]] += int(line[1])
                    
    sort_pos_fre = sorted(sort_pos_fre.items(), key=lambda d:d[1], reverse = True)

print ('END')

END


In [8]:
#查詢功能，包含：
#1.頻率前n個的詞性
#2.頻率前n個的詞彙
#3.頻率前n個且長度大於2的詞性
#4.頻率前n個且長度大於2的詞彙
#5.依詞性或詞彙做搜尋，可在之後加上數字表示要看幾位
#6.將查詢結果輸出

print_num = 20
global_set = []
request = ''

def print_out(temp_set,number): #印出的函式
    print ('排序總和為',sum([int(x[1]) for x in temp_set])) #全部累積(不代表是總數目，因為有包含可能)
    for i in range(number):
        print (i+1,temp_set[i][0],temp_set[i][1])
        
while(1):
    fd = input()
    find = ''
    find2 = 0
    if len(fd.split()) > 1:
        find = fd.split()[0]  
        find2 = int(fd.split()[1])
    else:
        find = fd
    
    clear_output()
    print ('find:',find,find2)
    
    if find == 'p': #將上一次搜尋結果輸出至桌面
        with codecs.open("C:\\Users\\user\\Desktop\\"+str(request)+".csv",'wb','utf8') as g:
            g.write('request:'+request+'\r\n') #印出上命令
            for i in global_set:
                g.write(i[0]+','+str(i[1])+'\r\n')
        print ('已輸出至桌面')
        continue
    
    temp_set = [] 
    
    if find == '1': #頻率前20個的詞性
        print_out(sort_pos_fre,print_num)
        temp_set = sort_pos_fre
    elif find == '2': #頻率前20個的詞彙
        print_out(sort_word_fre,print_num)
        temp_set = sort_word_fre 
    elif find == '3': #頻率前20個且長度大於2的詞性
        for i in sort_pos_fre:
            if len(i[0].split(')')) > 3:
                temp_set.append((i[0],i[1]))
        print_out(temp_set,print_num)
    elif find == '4': #頻率前20個且長度大於2的詞彙
        for i in sort_word_fre:
            if len(i[0].split(')')) > 3:
                temp_set.append((i[0],i[1]))
        print_out(temp_set,print_num)
    else: #依詞性或詞彙做搜尋，可在之後加上數字表示最少幾位
        if find in pos_word_set:
            temp = sorted(pos_word_set[find].items(), key=lambda d:d[1], reverse = True)
            if find2 == 0:
                print_out(temp,print_num)
                temp_set = temp
            else:
                for i in temp:
                    if len(i[0].split(')')) == find2+1:
                        temp_set.append((i[0],i[1]))
                print_out(temp_set,print_num)
        elif find in word_word_set:
            temp = sorted(word_word_set[find].items(), key=lambda d:d[1], reverse = True)
            if find2 == 0:
                print_out(temp,print_num)
                temp_set = temp
            else:
                for i in temp:
                    if len(i[0].split(')')) == find2+1:
                        temp_set.append((i[0],i[1]))
                print_out(temp_set,print_num)
        elif find == 'e': #離開
            break
        else:
            print ('輸入錯誤，請重新輸入')
    
    if len(temp_set) != 0:
        global_set = temp_set
        request = fd
print ('END')

find: e 0
END
